In [8]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import pandas as pd
import lightgbm as lgbm

### Read toxic and non-toxic data

In [9]:
toxic_df = pd.read_csv('/home/liujin/Offtarget_drugsafety/offtarget_application/Toxicity_prediction/toxic_predict_data/toxic_offtarget_profile.csv')
not_toxic_df = pd.read_csv('/home/liujin/Offtarget_drugsafety/offtarget_application/Toxicity_prediction/toxic_predict_data/nontoxic_offtarget_profile.csv')
toxic_df['label'] = 1
not_toxic_df['label'] = 0

df = pd.concat([toxic_df, not_toxic_df], axis=0)
print(toxic_df.shape, not_toxic_df.shape, df.shape)

data_df = df.drop(['smiles','label'], axis=1)
print(data_df.shape)

train_x, test_x, train_y, test_y = train_test_split(data_df, df['label'], test_size=0.2, random_state=999)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)
print(data_df.shape)
print(len(df['label']))

(877, 244) (1229, 244) (2106, 244)
(2106, 242)
(1684, 242) (422, 242) (1684,) (422,)
(2106, 242)
2106


### Model hyperparameter selection

In [10]:
import optuna  
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [7]:
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    # Parameter grid
    param_grid = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.1, 0.5, step=0.05), 
        "num_leaves": trial.suggest_int("num_leaves", 20, 1000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 500, step=100), 
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15, step=0.1),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0, step=0.1), 
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.9, step=0.1),
        "random_state": 2023,
    }

    X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.1,random_state=999) 
    
    model = lgbm.LGBMClassifier(objective="binary", **param_grid) 
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="auc", 
        callbacks=[
            LightGBMPruningCallback(trial, "auc"),
            lgbm.early_stopping(20),
        ],
    )
    
    preds = model.predict_proba(X_val)
    auc = roc_auc_score(y_val, preds[:, 1])
    return auc

In [11]:
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier") 
func = lambda trial: objective(trial, train_x, train_y)
study.optimize(func, n_trials=1000) 

[I 2023-11-21 14:08:51,594] A new study created in memory with name: LGBM Classifier


Training until validation scores don't improve for 20 rounds


[I 2023-11-21 14:09:04,079] Trial 0 finished with value: 0.8274606872451951 and parameters: {'n_estimators': 500, 'learning_rate': 0.35, 'num_leaves': 880, 'max_depth': 4, 'min_data_in_leaf': 100, 'lambda_l1': 95, 'lambda_l2': 20, 'min_gain_to_split': 3.4000000000000004, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 0 with value: 0.8274606872451951.


Early stopping, best iteration is:
[9]	valid_0's auc: 0.827461	valid_0's binary_logloss: 0.576194
[LightGBM] [Warning] lambda_l1 is set=90, reg_alpha=0.0 will be ignored. Current value: lambda_l1=90
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=10.5, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=10.5
[LightGBM] [Warning] lambda_l2 is set=100, reg_lambda=0.0 will be ignored. Current value: lambda_l2=100
Training until validation scores don't improve for 20 rounds


[I 2023-11-21 14:09:16,345] Trial 1 finished with value: 0.8353960396039604 and parameters: {'n_estimators': 1000, 'learning_rate': 0.2, 'num_leaves': 440, 'max_depth': 11, 'min_data_in_leaf': 400, 'lambda_l1': 90, 'lambda_l2': 100, 'min_gain_to_split': 10.5, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 1 with value: 0.8353960396039604.


...

[I 2023-11-21 14:31:16,414] Trial 97 pruned. Trial was pruned at iteration 0.


Training until validation scores don't improve for 20 rounds


[I 2023-11-21 14:31:51,629] Trial 98 finished with value: 0.9004076878276064 and parameters: {'n_estimators': 500, 'learning_rate': 0.5, 'num_leaves': 500, 'max_depth': 12, 'min_data_in_leaf': 100, 'lambda_l1': 5, 'lambda_l2': 10, 'min_gain_to_split': 1.3, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 74 with value: 0.9191904484566104.


Early stopping, best iteration is:
[18]	valid_0's auc: 0.900408	valid_0's binary_logloss: 0.394732
[LightGBM] [Warning] lambda_l1 is set=10, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10
[LightGBM] [Warning] bagging_fraction is set=0.9000000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9000000000000001
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=2.7, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.7
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
Training until validation scores don't improve for 20 rounds


[I 2023-11-21 14:31:57,867] Trial 99 pruned. Trial was pruned at iteration 4.


In [22]:
# print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest value (auc): {study.best_value:.5f}")
print(f"\tBest params:")
for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (auc): 0.91128
	Best params:
		n_estimators: 900
		learning_rate: 0.2
		num_leaves: 540
		max_depth: 6
		min_data_in_leaf: 100
		lambda_l1: 0
		lambda_l2: 0
		min_gain_to_split: 0.2
		bagging_fraction: 1.0
		bagging_freq: 1
		feature_fraction: 0.4


### Conduct five-fold cross-training based on the optimum hyperparameter

In [40]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, balanced_accuracy_score, matthews_corrcoef
import lightgbm as lgbm

# 创建 KFold 实例
kf = KFold(n_splits=5, random_state=999, shuffle=True)

train_x, test_x, train_y, test_y = train_test_split(data_df, df['label'], test_size=0.2, random_state=999)

# #实例化模型
# model = lgbm.LGBMClassifier(**study.best_params)
model = lgbm.LGBMClassifier(n_estimators=900,
                            learning_rate=0.2,
                            num_leaves=540,
                            max_depth=6,
                            min_data_in_leaf=100,
                            lambda_l1=0,
                            lambda_l2=0,
                            min_gain_to_split=0.2,
                            bagging_fraction=1.0,
                            bagging_freq=1,
                            feature_fraction=0.4,
                            random_state=2023)

acc_list = []
auc_list = []
f1_list = []
bacc_list = []
mcc_list = []
#对训练集进行五折交叉划分
for train_index,val_index in kf.split(train_x):
    X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
    y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(test_x)
    y_prob = model.predict_proba(test_x)

    acc_list.append(accuracy_score(test_y, y_pred))
    auc_list.append(roc_auc_score(test_y, y_prob[:,1]))
    f1_list.append(f1_score(test_y, y_pred, average='binary'))
    bacc_list.append(balanced_accuracy_score(test_y, y_pred))
    mcc_list.append(matthews_corrcoef(test_y, y_pred))

#计算五折交叉验证的平均值和方差
print('accuracy_score:', np.mean(acc_list), np.std(acc_list))
print('roc_auc_score:', np.mean(auc_list), np.std(auc_list))
print('f1_score:', np.mean(f1_list), np.std(f1_list))
print('balanced_accuracy_score:', np.mean(bacc_list), np.std(bacc_list))
print('matthews_corrcoef:', np.mean(mcc_list), np.std(mcc_list))


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=0.2, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.2
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_f

#### Save model performance results

In [39]:

result_df_offtarget = pd.DataFrame({'accuracy_score':acc_list,
                                'roc_auc_score':auc_list,
                                'f1_score':f1_list,
                                'balanced_accuracy_score':bacc_list,
                                'matthews_corrcoef':mcc_list})
# result_df_ecfp.head(5)
result_df_offtarget.to_csv('/home/liujin/Offtarget_drugsafety/offtarget_application/Toxicity_prediction/picture_draw/result_df_offtarget.csv',index=False)